In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "A7yb5zC1iHjj"
   },
   "source": [
    "# Displaying Rotated Anchor Boxes"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {
    "id": "MzNkG8IziHjm"
   },
   "outputs": [],
   "source": [
    "import numpy as np\n",
    "import tensorflow as tf\n",
    "from tensorflow import keras\n",
    "\n",
    "import matplotlib.pyplot as plt"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [],
   "source": [
    "class AnchorBox:\n",
    "    \"\"\"Generates anchor boxes.\n",
    "\n",
    "    This class has operations to generate anchor boxes for feature maps at\n",
    "    strides `[8, 16, 32, 64, 128]`. Where each anchor each box is of the\n",
    "    format `[x, y, width, height, phi]`.\n",
    "\n",
    "    Attributes:\n",
    "      aspect_ratios: A list of float values representing the aspect ratios of\n",
    "        the anchor boxes at each location on the feature map\n",
    "      scales: A list of float values representing the scale of the anchor boxes\n",
    "        at each location on the feature map.\n",
    "      num_anchors: The number of anchor boxes at each location on feature map\n",
    "      areas: A list of float values representing the areas of the anchor\n",
    "        boxes for each feature map in the feature pyramid.\n",
    "      strides: A list of float value representing the strides for each feature\n",
    "        map in the feature pyramid.\n",
    "      angles: A list of int values representing angles.\n",
    "    \"\"\"\n",
    "\n",
    "    def __init__(self):\n",
    "        self.aspect_ratios = [0.5, 1.0, 2.0]\n",
    "        #self.aspect_ratios = [1.0]\n",
    "        self.scales = [2 ** x for x in [0, 1 / 3, 2 / 3]]\n",
    "        #self.scales = [2 ** x for x in [0, 1 / 3]]\n",
    "\n",
    "        #adding in angle data\n",
    "        self.angles = [0, -45,]\n",
    "\n",
    "        self._num_anchors = len(self.aspect_ratios) * len(self.scales)\n",
    "        self._strides = [2 ** i for i in range(3, 8)]\n",
    "        self._areas = [x ** 2 for x in [32.0, 64.0, 128.0, 256.0, 512.0]]\n",
    "        self._anchor_dims = self._compute_dims()\n",
    "\n",
    "    def _compute_dims(self):\n",
    "        \"\"\"Computes anchor box dimensions for all ratios and scales at all levels\n",
    "        of the feature pyramid.\n",
    "        \"\"\"\n",
    "        anchor_dims_all = []\n",
    "        for area in self._areas:\n",
    "            anchor_dims = []\n",
    "            for ratio in self.aspect_ratios:\n",
    "                anchor_height = tf.math.sqrt(area / ratio)\n",
    "                anchor_width = area / anchor_height\n",
    "                dims = tf.reshape(\n",
    "                    tf.stack([anchor_width, anchor_height], axis=-1), [1, 1, 2]\n",
    "                )\n",
    "                for scale in self.scales:\n",
    "                    anchor_dims.append(scale * dims)\n",
    "            anchor_dims_all.append(tf.stack(anchor_dims, axis=-2))\n",
    "        return anchor_dims_all\n",
    "\n",
    "    def _get_anchors(self, feature_height, feature_width, level, theta):\n",
    "        \"\"\"Generates anchor boxes for a given feature map size and level\n",
    "\n",
    "        Arguments:\n",
    "          feature_height: An integer representing the height of the feature map.\n",
    "          feature_width: An integer representing the width of the feature map.\n",
    "          level: An integer representing the level of the feature map in the\n",
    "            feature pyramid.\n",
    "            \n",
    "            \n",
    "        Returns:\n",
    "          anchor boxes with the shape\n",
    "          `(feature_height * feature_width * num_anchors, 4)`\n",
    "        \"\"\"\n",
    "        rx = tf.range(feature_width, dtype=tf.float32) + 0.5\n",
    "        ry = tf.range(feature_height, dtype=tf.float32) + 0.5\n",
    "        centers = tf.stack(tf.meshgrid(rx, ry), axis=-1) * self._strides[level - 3]\n",
    "        centers = tf.expand_dims(centers, axis=-2)\n",
    "        centers = tf.tile(centers, [1, 1, self._num_anchors, 1])\n",
    "        dims = tf.tile(\n",
    "            self._anchor_dims[level - 3], [feature_height, feature_width, 1, 1]\n",
    "        )\n",
    "        anchors = tf.concat([centers, dims], axis=-1)\n",
    "        reshaped = tf.reshape(\n",
    "            anchors, [feature_height * feature_width * self._num_anchors, 4]\n",
    "        )\n",
    "        test = reshaped.numpy()\n",
    "        return reshaped\n",
    "\n",
    "    def get_anchors(self, image_height, image_width):\n",
    "        \"\"\"Generates anchor boxes for all the feature maps of the feature pyramid.\n",
    "\n",
    "        Arguments:\n",
    "          image_height: Height of the input image.\n",
    "          image_width: Width of the input image.\n",
    "\n",
    "        Returns:\n",
    "          anchor boxes for all the feature maps, stacked as a single tensor\n",
    "            with shape `(total_anchors, 4)`\n",
    "        \"\"\"\n",
    "        anchors = [\n",
    "            self._get_anchors(\n",
    "                tf.math.ceil(image_height / 2 ** i),\n",
    "                tf.math.ceil(image_width / 2 ** i),\n",
    "                i,\n",
    "                self.angles,\n",
    "            )\n",
    "            for i in range(3, 8)\n",
    "        ]\n",
    "        return tf.concat(anchors, axis=0)\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [],
   "source": [
    "def visualize_detections(\n",
    "    image, boxes, figsize=(7, 7), linewidth=1, color=[0, 0, 1]\n",
    "):\n",
    "    \"\"\"Visualize Detections\"\"\"\n",
    "    image = np.array(image, dtype=np.uint8)\n",
    "    plt.figure(figsize=figsize)\n",
    "    plt.axis(\"off\")\n",
    "    plt.imshow(image)\n",
    "    ax = plt.gca()\n",
    "    for box in boxes:\n",
    "        x1, y1, x2, y2 = box\n",
    "        w, h = x2 - x1, y2 - y1\n",
    "        patch = plt.Rectangle(\n",
    "            [x1, y1], w, h, fill=False, edgecolor=color, linewidth=linewidth\n",
    "        )\n",
    "        ax.add_patch(patch)\n",
    "    plt.show()\n",
    "    return ax"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "2022-04-24 20:43:19.483322: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-04-24 20:43:19.483636: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-04-24 20:43:19.490227: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-04-24 20:43:19.490510: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-04-24 20:43:19.490836: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-04-24 20:43:19.491039: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-04-24 20:43:19.491930: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA\n",
      "To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.\n",
      "2022-04-24 20:43:19.670036: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-04-24 20:43:19.670267: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-04-24 20:43:19.670462: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-04-24 20:43:19.670642: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-04-24 20:43:19.670821: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-04-24 20:43:19.671001: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-04-24 20:43:20.352310: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-04-24 20:43:20.352590: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-04-24 20:43:20.352788: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-04-24 20:43:20.352979: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-04-24 20:43:20.353170: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-04-24 20:43:20.353352: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 35 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1070, pci bus id: 0000:0d:00.0, compute capability: 6.1\n",
      "2022-04-24 20:43:20.353838: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-04-24 20:43:20.353989: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 5196 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1060 6GB, pci bus id: 0000:0c:00.0, compute capability: 6.1\n",
      "2022-04-24 20:43:20.358455: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate 35.12M (36831232 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory\n",
      "2022-04-24 20:43:20.359295: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate 31.61M (33148160 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory\n"
     ]
    }
   ],
   "source": [
    "boxes = AnchorBox()\n",
    "img = tf.zeros([100, 100, 3])\n",
    "\n",
    "anchors = boxes.get_anchors(100, 100)\n",
    "\n",
    "anchor_dims = boxes._compute_dims()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "iVBORw0KGgoAAAANSUhEUgAAAYoAAAGKCAYAAAASfgYQAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjUuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/YYfK9AAAACXBIWXMAAAsTAAALEwEAmpwYAAAJeUlEQVR4nO3d0XLcxhFA0WEq///J2TypSpDpawGzO8BY5zzaaA4tJbq1AtH4er1eAwD+zn/u/gYAeDahACAJBQBJKABIQgFA+m/9y6+vLz8SBfCHeL1eX9/9c58oAEhCAUASCgCSUACQhAKAJBQAJKEAIAkFAEkoAEhCAUASCgCSUACQhAKAJBQAJKEAIAkFAClfXPQe/xtjfPsuDAA2sCAUX+O+ULxuOvt3z/3E9/fja1752jPfzzv+W65+jV/nPvnr+k//7J1f/x3Xvtu7fo9Wz894+p8j7z7zr/zVEwBJKABIQgFAEgoAklAAkIQCgCQUACShACAJBQBJKABIQgFAEgoAklAAkIQCgPT1en2/VnaMMb6+vv7+X/62d6wXBuDTXq/v/7Be8D6Kd7jzHQmfPPcT7zjY8X0UM9/zd2fv8D6Kme/5znc7zPwejRvOfcfXWfHr9en3nfwu76MA4AKhACAJBQBJKABIQgFAEgoAklAAkIQCgCQUACShACAJBQBJKABIQgFAWrRmHICnu3nN+G4rkWdZM35t1prx69/LzNkrZq0ZP3+dNeMAbEIoAEhCAUASCgCSUACQhAKAJBQAJKEAIAkFAEkoAEhCAUASCgCSUACQFq0Z323T5Y9ZgD/HzWvGd7XjamFrxj+zstua8bnZu9eMP/3/F9aMA7AxoQAgCQUASSgASEIBQBIKAJJQAJCEAoAkFAAkoQAgCQUASSgASEIBQFq0ZhyAp7t5zfhuK5FXzVozfpy1Zvza7OzZK2atGT9/nTXjAGxCKABIQgFAEgoAklAAkIQCgCQUACShACB5MhuAMYYnsx8668ns46wns6/Nzp69YtaT2eev82Q2AJsQCgCSUACQhAKAtOinnmZvQgHwaTf/1NOsJ//k0sysn3o6zvqpp2uzs2evmPVTT+ev81NPAGxCKABIQgFAEgoAklAAkIQCgCQUACShACAJBQBJKABIQgFAEgoAklAAkBatGQfg6W5eM77bSuRVs9aMH2etGb82O3v2illrxs9f95w14z5RADDG8IniobM+URxnfaK4Njt79opZnyjOX/ecTxRuZgOQhAKAJBQAJKEAIAkFAEkoAEhCAUASCgCSUACQhAKAJBQAJEsBARhjWAr40FlLAY+zlgJem509e8WspYDnr7vz9+nIXz0BkIQCgCQUACShACAJBQBJKABIQgFAEgoAklAAkIQCgCQUACShACAJBQDJmnEAxhjWjD901prx46w149dmZ89eMWvN+PnrrBkHYBNCAUASCgCSUACQhAKAJBQAJKEAIAkFAEkoAEhCAUASCgCSUACQhAKAtGjN+OzWRwA+7eY147OevCp8Ztaa8eOsNePXZmfPXjFrzfj5656zZtyLi3iQ2T8MgBleXPTIWZ8ojrM+UVybnT17xaxPFOeve84nCjezAUhCAUASCgCSUACQhAKAJBQAJKEAIAkFAEkoAEhCAUASCgCSUACQbI8FYIxhe+xDZ22PPc7aHnttdvbsFbO2x56/zvZYADYhFAAk9ygAGGO4R/HQWfcojrPuUVybnT17xax7FOevc48CgE0IBQBJKABIQgFAEgoAklAAkIQCgCQUACShACAtWuGx21OkP2YB/hw3r/DY1W6P7f88/9RVBTVrhce12dmzV8xa4XH+Ois8ANiEUACQhAKAJBQAJKEAIAkFAEkoAEhCAUBa9GQ27G72gS94vpufzN7tKdJVs57MPs56Mvva7OzZK2Y9mX3+Ok9mA7AJoQAgCQUAyc1sAMYYbmY/dNbN7OOsm9nXZmfPXjHrZvb569zMBmATQgFAEgoAklAAkIQCgCQUACShACAJBQBJKABIQgFAEgoAklAAkIQCgLRozfjs1kcAPu3mNeOznrwqfGbWmvHjrDXj12Znz14xa834+eusGQdgE0IBQPIqVADGGLffo9jt71NXzbpHcZx1j+La7OzZK2bdozh/nXsUAGxCKABIQgFAEgoAklAAkIQCgCQUACShACAJBQDJCg8AxhhWeDx01gqP46wVHtdmZ89eMWuFx/nrrPAAYBNCAUASCgCSUACQhAKAJBQAJKEAIAkFAEkoAEhCAUASCgCSUACQhAKAJBQApEXvo5hdDwzAp938PopZT36nxMys91EcZ72P4trs7NkrZr2P4vx13kcBwCaEAoAkFAAkoQAgCQUASSgASEIBQBIKAJJQAJCEAoAkFAAkoQAgCQUAadGacQCe7uY147utRF41a834cdaa8Wuzs2evmLVm/Px11owDsAmhACAJBQBJKABIQgFAEgoAklAAkIQCgCQUACShACAJBQBJKABIQgFAsmYcgDGGNeMPnbVm/Dhrzfi12dmzV8xaM37+OmvGAdiEUACQhAKAJBQAJKEAIAkFAEkoAEhCAUASCgCSUACQhAKAJBQAJKEAIFkzDsAYw5rxh85aM36ctWb82uzs2StmrRk/f5014wBsQigASEIBQBIKAJJQAJCEAoAkFAAkoQAgCQUASSgASEIBQBIKAJJQAJCEAoC06H0Uu61E/jEL8Oe4+X0Uu9ptB/3P80/du1+z3kdxbXb27BWz3kdx/rrnvI9ik08UAO8w++fRv9vmnyie/Ja6mVmfKI6zPlFcm509e8WsTxTnr3vOJwo3swFIQgFAEgoAklAAkIQCgCQUACShACAJBQBJKABIQgFAEgoA0qKlgAA83c1LAXdbYLZq1lLA46ylgNdmZ89eMWsp4PnrLAUEYBNCAUASCgCSUACQhAKAJBQAJKEAIAkFAEkoAEhCAUASCgCSUACQhAKAtGjN+OzWRwA+7eY147OevCp8Ztaa8eOsNePXZmfPXjFrzfj566wZB2ATQgFAEgoAklAAkIQCgCQUACShACAJBQDJk9kAjDE8mf3QWU9mH2c9mX1tdvbsFbOezD5/nSezAdiEUACQhAKAtOhmNgBPd/PN7N1uvK2adTP7OOtm9rXZ2bNXzLqZff46N7MB2IRQAJCEAoAkFAAkoQAgCQUASSgASEIBQBIKAJJQAJCEAoAkFAAkoQAgbfIqVNaZWQv/jtmrX+PXuTNf53ev/e66d/56feJ7/sT8jrM/z1/5Oiu+70//7+s6oeAX1oz//nXWjJ+bGzec++u8NeP/PPdX/uoJgCQUACShACAJBQBJKABIQgFAEgoAklAAkIQCgCQUACShACAJBQDp6/V634ZBAP59fKIAIAkFAEkoAEhCAUASCgCSUACQ/g/G7tcAGpDMHAAAAABJRU5ErkJggg==\n",
      "text/plain": [
       "<Figure size 504x504 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    },
    {
     "data": {
      "text/plain": [
       "<AxesSubplot:>"
      ]
     },
     "execution_count": 5,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "visualize_detections(img, anchors)"
   ]
  }
 ],
 "metadata": {
  "colab": {
   "collapsed_sections": [],
   "name": "retinanet",
   "provenance": [],
   "toc_visible": true
  },
  "interpreter": {
   "hash": "61c63460f299b72050a017299847de40e48391aa12573d6685850838867f4476"
  },
  "kernelspec": {
   "display_name": "RLModel",
   "language": "python",
   "name": "rlmodel"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}